In [6]:
from ctypes import *
import numpy as np
import sys

In [7]:
# The computational routine
def read_bfee(inBytes):
    array = {}

    array["timestamp_low"] = c_double(inBytes[0] + (inBytes[1] << 8) +(inBytes[2] << 16) + (inBytes[3] << 24)).value
    array["bfee_count"] = c_double(inBytes[4] + (inBytes[5] << 8)).value
    array["Nrx"] = inBytes[8]
    array["Ntx"] = inBytes[9]
    array["rssi_a"] = c_double(inBytes[10]).value
    array["rssi_b"] = c_double(inBytes[11]).value
    array["rssi_c"] = c_double(inBytes[12]).value
    array["noise"] = c_byte(inBytes[13]).value
    array["agc"] = c_double(inBytes[14]).value   
    antenna_sel = inBytes[15]        
    length = inBytes[16] + (inBytes[17] << 8)
    array["rate"] = c_double(inBytes[18] + (inBytes[19] << 8)).value
    calc_len = int((30 * (array["Nrx"] * array["Ntx"] * 8 * 2 + 3) + 7) / 8)
    payload = inBytes[20:]
    csi = np.empty((array["Ntx"], array["Nrx"], 30), dtype=np.complex)
    csi[:] = np.nan
    array["perm"] = []

    # Check that length matches what it should 
    if (length != calc_len):
        raise Exception("Wrong beamforming matrix size.")
    
    # Compute CSI from all this crap :)
    index = 0
    for i in range(30):
        index += 3
        remainder = index % 8
        for j in range(array["Ntx"]):
                for k in range(array["Nrx"]):
                    real = c_byte((payload[int(index / 8)]) >> remainder).value | c_byte((payload[int(index / 8 + 1)]) << (8-remainder)).value
                    complx = c_byte((payload[int(index / 8 + 1)]) >> remainder).value | c_byte((payload[int(index / 8 + 2)]) << (8-remainder)).value            
                    csi[k, j, i] = complex(real, complx)
                    index += 16;
        csi[:, :, i] = np.flip(csi[:, :, i], 1)
        
    array["perm"].append((antenna_sel) & 0x3) 
    array["perm"].append((antenna_sel >> 2) & 0x3)
    array["perm"].append((antenna_sel >> 4) & 0x3) 
    array["csi"] = csi
    
    return array
    
def read_bf_file(filename):
    f = open(filename, "rb")
    
    #get length of file
    f.seek(0, 2)
    len = f.tell()
    f.seek(0, 0)

    # Initialize variables
    ret = []               #Holds the return values - 1x1 CSI is 95 bytes big, so this should be upper bound
    cur = 0                #Current offset into file
    count = 0              #Number of records output
    broken_perm = 0        #Flag marking whether weve encountered a broken CSI yet       
    triangle = [0, 2, 5]   #What perm should sum to for 1,2,3 antennas            
    
    # Process all entries in file
    # Need 3 bytes -- 2 byte size field and 1 byte code
    while (cur < (len-3)):
        #Read size and code
        field_len = int.from_bytes(f.read(2), byteorder='big')
        code = int.from_bytes(f.read(1), byteorder='little')
        cur += 3
        
        #If unhandled code, skip (seek over) the record and continue
        if(code == 187): # get beamforming or phy data
            byts = f.read(field_len-1)
            cur += field_len-1 
        else: #skip all other info
            f.seek(field_len-1, 1)     
            cur += field_len-1
            continue
            
        if (code == 187): #hex2dec('bb')) Beamforming matrix -- output a record
            count += 1
            ret.append(read_bfee(byts))
            perm = ret[count-1]["perm"]
            Nrx = ret[count-1]["Nrx"]
                                   
            if (Nrx == 1): # No permuting needed for only 1 antenna
                continue
            
            if (sum(perm) != triangle[Nrx-1]): # matrix does not contain default values
                if (broken_perm == 0):
                    broken_perm = 1
            else:
                ret[count-1]["csi"][:, perm[:Nrx-1], :] = ret[count-1]["csi"][:, :Nrx-1, :]   
            
    # Close file
    f.close()
    
    return ret[:count]